In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
df_raw = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')

In [ ]:
df_raw

In [ ]:
target = 'class'

# Analize features

In [ ]:
df_raw.dtypes

In [ ]:
df_raw.describe().T

In [ ]:
df_raw.isna().sum()

In [ ]:
for col in df_raw.columns:
    print(df_raw[col].value_counts())

There seem to be no missing data. All features appear to be well characterized.

The feature `stalk-color-below-ring` could be dropped since it has a sole value for the whole dataset.

# Transform target

In [ ]:
df_transformed = df_raw.copy()

In [ ]:
df_transformed[target].value_counts(dropna=False)

In [ ]:
(df_transformed[target] == 'e').value_counts()

In [ ]:
df_transformed[target] = (df_transformed[target] == 'e')

# Split dataset

In [ ]:
 from sklearn.model_selection import train_test_split

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_transformed.drop(columns=target), df_transformed[target])

# Run Catboost Model

In [ ]:
clf = CatBoostClassifier(
    cat_features=X_train.columns,
    iterations=10,
    depth=4,
    learning_rate=1,
    loss_function='Logloss',
    verbose=True
)

In [ ]:
y_train.value_counts()

In [ ]:
clf.fit(X_train, y_train.astype(int))

In [ ]:
from sklearn.model_selection import KFold, cross_validate, cross_val_score

In [ ]:
cross_val_score( clf, X_train, y_train.astype(int), cv=5, scoring='accuracy')

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
pd.crosstab(y_pred, y_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, y_pred)